## Import Necessary Libraries

In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import dask.dataframe as dd

from dask.distributed import Client, LocalCluster
from statsmodels.tsa.stattools import adfuller
from sklearn.linear_model import SGDRegressor

import warnings
warnings.filterwarnings("ignore")

%matplotlib inline

## Loading data

In [7]:
df = pd.read_csv('../../3_Data/processed/2025_hourly_all_clean.csv')
df.head()

,tpep_pickup_datetime,passenger_demand,taxi_demand
0,2025-01-01 00:00:00,9132,7344
1,2025-01-01 01:00:00,8996,8468
2,2025-01-01 02:00:00,7364,7257
3,2025-01-01 03:00:00,4904,4915
4,2025-01-01 04:00:00,3015,2918


In [8]:
df.tail()

,tpep_pickup_datetime,passenger_demand,taxi_demand
6566,2025-09-30 19:00:00,9595,9779
6567,2025-09-30 20:00:00,8882,9539
6568,2025-09-30 21:00:00,9048,9965
6569,2025-09-30 22:00:00,7026,8001
6570,2025-09-30 23:00:00,3984,4587


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6571 entries, 0 to 6570
Data columns (total 3 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   tpep_pickup_datetime  6571 non-null   object
 1   passenger_demand      6571 non-null   int64 
 2   taxi_demand           6571 non-null   int64 
dtypes: int64(2), object(1)
memory usage: 154.1+ KB


In [10]:
df.describe()

,passenger_demand,taxi_demand
count,6571.000000,6571.000000
mean,5407.537513,5449.311520
std,3190.897504,3055.996053
min,0.000000,0.000000
25%,2208.500000,2574.500000
50%,6037.000000,6016.000000
75%,7957.000000,7582.000000
max,12923.000000,14039.000000


In [15]:
df.loc[0, 'tpep_pickup_datetime'].weekofyear

1

In [16]:
df.loc[0, 'tpep_pickup_datetime'].dayofyear

1

In [18]:
# let's set "tpep_pickup_datetime" as index
data = df.set_index('tpep_pickup_datetime').copy()
data

,passenger_demand,taxi_demand
tpep_pickup_datetime,,
2025-01-01 00:00:00,9132,7344
2025-01-01 01:00:00,8996,8468
2025-01-01 02:00:00,7364,7257
2025-01-01 03:00:00,4904,4915
2025-01-01 04:00:00,3015,2918
...,...,...
2025-09-30 19:00:00,9595,9779
2025-09-30 20:00:00,8882,9539
2025-09-30 21:00:00,9048,9965


In [19]:
data.index.month

Index([1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       ...
       9, 9, 9, 9, 9, 9, 9, 9, 9, 9],
      dtype='int32', name='tpep_pickup_datetime', length=6571)

In [20]:
def add_timed_features(df: pd.DataFrame) -> pd.DataFrame:
    df['hour'] = df.index.hour
    df['day'] = df.index.day
    df['month'] = df.index.month
    df['dayofweek'] = df.index.dayofweek
    return df

In [21]:
add_timed_features(data)

,passenger_demand,taxi_demand,hour,day,month,dayofweek
tpep_pickup_datetime,,,,,,
2025-01-01 00:00:00,9132,7344,0,1,1,2
2025-01-01 01:00:00,8996,8468,1,1,1,2
2025-01-01 02:00:00,7364,7257,2,1,1,2
2025-01-01 03:00:00,4904,4915,3,1,1,2
2025-01-01 04:00:00,3015,2918,4,1,1,2
...,...,...,...,...,...,...
2025-09-30 19:00:00,9595,9779,19,30,9,1
2025-09-30 20:00:00,8882,9539,20,30,9,1
2025-09-30 21:00:00,9048,9965,21,30,9,1
